In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.optim as optim



In [3]:


#Simple data loading for usage in
#the rest of the training.
#Before using for the first time set download to
#true

apply_flip_percentage = 0.4
apply_affine_percentage = 0.8
CIFAR_10_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_10_STD_DEV = [0.24703233, 0.24348505, 0.26158768]
MEAN = CIFAR_10_MEAN
STD = CIFAR_10_STD_DEV

 

toTensor = transforms.ToTensor()
normalize = transforms.Normalize(MEAN, STD)
flipTransform = transforms.RandomHorizontalFlip(p=apply_flip_percentage)
affineTransform = transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=0.5)

 

trainTransform = transforms.Compose([flipTransform, transforms.RandomApply([affineTransform], p=apply_affine_percentage), toTensor, normalize])
validTransform = transforms.Compose([toTensor, normalize])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=trainTransform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=validTransform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
class residual_block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(residual_block, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=3, stride=stride, padding=1
        )
        self.bn1 = nn.BatchNorm2d(in_channels)
        
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * 4,
            kernel_size=1,
            stride=1,
            padding=0,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv1(x)


        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        
        return x

In [11]:


class ResNet(nn.Module):
  """A simple model for generating a residual convolutional neural network.
  """
  def __init__(self, residual_block, layers, image_channels=3, num_classes=10):
      super(ResNet, self).__init__()
      self.in_channels = 64

      self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
      self.bn1 = nn.BatchNorm2d(64)
      self.relu = nn.ReLU()
      self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

      self.layer1 = self._make_layer(
          residual_block, layers[0], intermediate_channels=64, stride=1
      )
      self.layer2 = self._make_layer(
          residual_block, layers[1], intermediate_channels=128, stride=2
      )
      self.layer3 = self._make_layer(
          residual_block, layers[2], intermediate_channels=256, stride=2
      )

      self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
      self.fc = nn.Linear(256 * 4, num_classes)

    # parametri su već inicijalizirani pozivima Conv2d i Linear
    # ali možemo ih drugačije inicijalizirati
  def _make_layer(self, residual_block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            residual_block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        self.in_channels = intermediate_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(residual_block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)
  
  def forward(self, x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      x = self.maxpool(x)

      x = self.layer1(x)
      x = self.layer2(x)
      x = self.layer3(x)

      x = self.avgpool(x)
      x = x.reshape(x.shape[0], -1)
      x = self.fc(x)


      return x

In [13]:
def ResNet14():
    return ResNet(residual_block, [2, 2, 2])


def ResNet28(img_channel=3, num_classes=10):
    return ResNet(residual_block, [3, 4, 6])

In [8]:



def evaluate(net, type):
    """
    Performs the evaluation of the current performance of a
    given convolutional network. It can perform the evaluation on 
    both training and testing sets. Standard evaluation metrics are
    calcualted such as, accuracy and confusion matrix.
    Parameters
    ----------
    net: ConvolutionalModel
        ConvNet whose performance needs to be evaluated.
    type: bool
        True if eval is made on testing set, false otherwise
    Return
    ------
    loss
        Current loss on the chosen set
    accuracy
        Current acc on the chosen set
    """
    device = torch.device('cuda')
    total = 0
    correct = 0
    confMatrix = np.zeros((10, 10), int)
    lossFunc = nn.CrossEntropyLoss()
    accLoss = 0
    if type:
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images = images.to(device=device)
                labels = labels.to(device=device)

                output = net.forward(images)
                loss = lossFunc(output, labels)
                _, predictions = torch.max(output.data, 1)
                total += labels.size(0)
                accLoss += loss.item()
                correct += (predictions == labels).sum().item()
                for j in range(labels.size(0)):
                    confMatrix[predictions[j], labels[j]] += 1
    else:
        with torch.no_grad():
            for data in trainloader:
                images, labels = data
                images = images.to(device=device)
                labels = labels.to(device=device)

                output = net.forward(images)
                loss = lossFunc(output, labels)
                _, predictions = torch.max(output.data, 1)
                total += labels.size(0)
                accLoss += loss.item()
                correct += (predictions == labels).sum().item()
                for j in range(labels.size(0)):
                    confMatrix[predictions[j], labels[j]] += 1

    print("Accuracy of the neural network on CIFAR_10 is: %.2f %%" %((correct/total)*100))
    #print(classes)
    #print(confMatrix)
    #specificMetrics(confMatrix)
    return (accLoss/(total/trainloader.batch_size)), (correct/total)

def specificMetrics(confMatrix):
    """
    Calculates precision and recall from a given confusion
    matrix and prints calculated metrics.
    Parameters
    ----------
    confMatrix: n x n numpy array
        Made from the predictions and true labels of a
        given set of data
    """
    for i in range(np.size(confMatrix, 0)):
        print("Class: " + classes[i], end='')
        precc = 0
        recal = 0
        tp = 0
        fp = 0
        fn = 0
        for j in range(np.size(confMatrix, 0)):
            if i == j:
                tp += confMatrix[i, j]
            else:
                fn += confMatrix[j, i]
                fp += confMatrix[i, j]
            
        precc = tp/(tp + fp)
        recal = tp/(tp + fn)

        print(", Precision: %.2f, Recall: %.2f" %(precc, recal))

In [9]:



def plot_training_progress(save_dir, data,filename):
  fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16,8))

  linewidth = 2
  legend_size = 10
  train_color = 'm'
  val_color = 'c'

  num_points = len(data['train_loss'])
  x_data = np.linspace(1, num_points, num_points)
  ax1.set_title('Cross-entropy loss')
  ax1.plot(x_data, data['train_loss'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax1.plot(x_data, data['valid_loss'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax1.legend(loc='upper right', fontsize=legend_size)
  ax2.set_title('Average class accuracy')
  ax2.plot(x_data, data['train_acc'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
  ax2.plot(x_data, data['valid_acc'], marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
  ax2.legend(loc='upper left', fontsize=legend_size)
  ax3.set_title('Learning rate')
  ax3.plot(x_data, data['lr'], marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='learning_rate')
  ax3.legend(loc='upper left', fontsize=legend_size)

  save_path = os.path.join(save_dir, filename+'.png')
  print('Plotting in: ', save_path)
  plt.savefig(save_path)

In [16]:


def trainNetwork(network, name, lr, gamma):
    """Performs a standard procedure for training a neural network.
    Training progress after each learning epoch is evaluated in order to
    gain insigth into ResNets continuous performance.
    Important notes
    ---------------
    Loss function: Cross entropy loss
    Optimizer: SGD
    
    Scheduler: ExponentialLR
    """
    plot_data = {}
    plot_data['train_loss'] = []
    plot_data['valid_loss'] = []
    plot_data['train_acc'] = []
    plot_data['valid_acc'] = []
    plot_data['lr'] = []
    SAVE_DIR = '.'
    device = torch.device('cuda')

    net = network.to(device=device)

    lossFunc = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr, weight_decay=0.0005)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma)

    epoch = 50
    

    for e in range(epoch):
    
        accLoss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)

            optimizer.zero_grad()

            outputs = net.forward(inputs)
            loss = lossFunc(outputs, labels)
            loss.backward()
            optimizer.step()

            accLoss += loss.item()

            if i % 100 == 0:
                print("Epoch: %d, Iteration: %5d, Loss: %.3f" % ((e + 1), (i), (accLoss / (i + 1))))
                
        train_loss, train_acc = evaluate(net, False)
        val_loss, val_acc = evaluate(net, True)

        plot_data['train_loss'] += [train_loss]
        plot_data['valid_loss'] += [val_loss]
        plot_data['train_acc'] += [train_acc]
        plot_data['valid_acc'] += [val_acc]
        plot_data['lr'] += [scheduler.get_last_lr()]

        scheduler.step()

    plot_training_progress(SAVE_DIR, plot_data,'_cifar10_resnet')
    PATH = './'+name+'_cifar10_resnet.pth'
    torch.save(net.state_dict(), PATH)

trainNetwork(ResNet14(),"ResNet14",0.1,0.99)

KeyboardInterrupt: ignored